# Data cleaning


In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
import tldextract
import re
from urllib.parse import urlparse

def clean_extract_domain(url):
    """Cleans, standardizes URLs, and extracts the domain."""
    if pd.isna(url):
        return None
    url = re.sub(r'^(?:http:\/\/|https:\/\/)?(?:www\.)?', '', url)
    url = url.strip("/").strip()  # Clean trailing or leading slashes and spaces
    url = url.rstrip('/')
    try:
        parsed_url = urlparse('http://' + url)  # Ensure scheme is present for parsing
        domain = parsed_url.netloc
        domain = re.sub(r'^www\.', '', domain)  # Remove 'www.'
        return domain
    except Exception as e:
        print(f"Error parsing URL '{url}': {e}")
        return None

def extract_domain_features(domain):
    """Extracts various features from a domain."""
    if not domain:
        return {'domain_length': 0, 'unique_characters': 0, 'digit_count': 0,
                'letter_count': 0, 'hyphen_count': 0, 'dot_count': 0, 'exclamation_count': 0,
                'question_count': 0}
    extracted = tldextract.extract(domain)
    return {
        'domain_length': len(domain),
        'unique_characters': len(set(domain)),
        'digit_count': sum(c.isdigit() for c in domain),
        'letter_count': sum(c.isalpha() for c in domain),
        'hyphen_count': domain.count('-'),
        'dot_count': domain.count('.'),
        'exclamation_count': domain.count('!'),
        'question_count': domain.count('?')
    }

def load_data(dataset_query_path, search_results_paths):
    """
    Loads and merges the query dataset with multiple search results datasets.

    Parameters:
    - dataset_query_path: The file path to the dataset containing the queries and the correct URLs.
    - search_results_paths: A list of file paths to the datasets containing search results.

    Returns:
    - A merged DataFrame containing all the data.
    """
    # Load the dataset containing queries and correct URLs
    dataset_incl_query = pd.read_csv(dataset_query_path)
    dataset_incl_query.drop(columns=['Unnamed: 0'], inplace=True)  # Drop unnecessary columns

    # Initialize a DataFrame to hold the merged search results
    merged_search_results = pd.DataFrame()

    # Load and concatenate each search results dataset
    for path in search_results_paths:
        search_results = pd.read_csv(path)

        if merged_search_results.empty:
            merged_search_results = search_results
        else:
            # Ensure that the DataFrame structures align before concatenating
            merged_search_results = pd.concat([merged_search_results, search_results], axis=0, ignore_index=True)

    # Merge the query dataset with the concatenated search results
    merged_dataset = pd.merge(dataset_incl_query, merged_search_results, on='EntityNumber', how='left')

    return merged_dataset


def get_core_domain(url):
    """Extracts the core registrable domain from a URL using tldextract."""
    if pd.isna(url):
        return None
    try:
        extracted = tldextract.extract(url)
        if extracted.domain and extracted.suffix:  # Ensure domain and suffix are not empty
            return "{}.{}".format(extracted.domain, extracted.suffix).lower()
        return None
    except Exception as e:
        print(f"Error extracting domain from URL '{url}': {str(e)}")
        return None

def preprocess_domains(dataframe, url_columns):
    """Preprocess and attach core domains for given URL columns in a DataFrame."""
    for col in url_columns:
        dataframe[f'{col}_domain'] = dataframe[col].apply(get_core_domain)

def compare_domains(row, main_domain_col, comparison_domain_cols):
    """Compares main domain with a list of other domains and assigns numerical identifiers for matches, or -1 if no match."""
    matches = []
    main_domain = row[main_domain_col]
    for index, col in enumerate(comparison_domain_cols):
        comparison_domain = row[col]
        if main_domain and comparison_domain and main_domain == comparison_domain:
            matches.append(index + 1)  # Start at 1 to correspond to URL1, URL2, etc.

    return matches if matches else [-1]  # Return matches or -1 if no matches

# Create 'Abbreviation' from the first letter of each word in 'OfficialName'
# only if 'Abbreviation' column is empty
def create_abbreviation(name):
    if pd.isna(name):
        return ""
    return ''.join(word[0] for word in name.split())

def get_domain_without_tld(url):
    if pd.isna(url):
        return ""  # Return an empty string or None, as preferred
    extracted = tldextract.extract(url)
    return extracted.domain  # Return only the domain part without the suffix

# Function to check if any word from 'OfficialName' is in the URL
def check_words_in_url(words, url):
    if pd.isna(words) or pd.isna(url):
        return False
    words = words.split()
    return any(word in url for word in words)

# Function to check if 'Abbreviation' is in the URL
def check_abbreviation_in_url(abbreviation, url):
    if pd.isna(abbreviation) or pd.isna(url):
        return False
    return abbreviation in url

# Load data and preprocess domains
dataset_query_path = 'dataset_incl_query.csv'
search_results_paths = ['search_results_DDG.csv']
merged_dataset = load_data(dataset_query_path, search_results_paths)

# Lowercase all URL columns before processing
url_columns = ['URL'] + [f'URL{i}' for i in range(1, 6)]
for col in url_columns:
    merged_dataset[col] = merged_dataset[col].str.lower()


# Initialize 'OfficialName_cleaned' and perform cleaning operations
merged_dataset['OfficialName_cleaned'] = merged_dataset['OfficialName'].str.lower()
merged_dataset['OfficialName'] = merged_dataset['OfficialName'].str.lower() # Lowercase the 'OfficialName' column
merged_dataset['OfficialName_cleaned'] = merged_dataset['OfficialName_cleaned'].str.replace(r"\[.*?\]", "", regex=True) # Remove text within brackets (including the brackets)
merged_dataset['OfficialName_cleaned'] = merged_dataset['OfficialName_cleaned'].str.replace(r"\(.*?\)", "", regex=True)
merged_dataset['OfficialName_cleaned'] = merged_dataset['OfficialName_cleaned'].str.replace('-', '', regex=True) # Remove hyphens from 'OfficialName'
merged_dataset['Abbreviation'] = merged_dataset.apply(
    lambda row: create_abbreviation(row['OfficialName_cleaned']) if pd.isna(row['Abbreviation']) else row['Abbreviation'],
    axis=1
    ) # Abbreviation creation if none is given


# Define the URL columns you are working with, e.g., 'URL', 'URL1', 'URL2', etc.
url_columns = ['URL'] + [f'URL{i}' for i in range(1, 6)]
url_columns_without_official =[f'URL{i}' for i in range(1, 6)]
# Preprocess the domains to extract the core domain
preprocess_domains(merged_dataset, url_columns)

# Extract and append domain features for each URL column with a suffix indicating the column
for col in url_columns:
    # Extract clean domain first
    merged_dataset[f'{col}_clean_domain'] = merged_dataset[col].apply(clean_extract_domain)


# Compare domains and add the comparison results to the DataFrame
comparison_domain_cols = [f'{col}_clean_domain' for col in url_columns if col != 'URL']
merged_dataset['domain_matches'] = merged_dataset.apply(
    lambda row: compare_domains(row, 'URL_clean_domain', comparison_domain_cols), axis=1
)

# Calculate the length of each 'URL(i)_domain'
for col in url_columns_without_official:
    domain_col = f'{col}_domain'
    if domain_col in merged_dataset.columns:
        # Apply the extraction of domain without TLD
        merged_dataset[f'{col}_core_domain'] = merged_dataset[domain_col].apply(get_domain_without_tld)
        # Calculate the length of the domain without TLD
        merged_dataset[f'{col}_domain_length'] = merged_dataset[f'{col}_core_domain'].str.len()

merged_dataset['Abbreviation'] = merged_dataset['Abbreviation'].str.lower() # Lowercase the 'OfficialName' column

for col in url_columns:
    if f'{col}_clean_domain' in merged_dataset.columns:
        # Check if any word from 'OfficialName' is in the URL
        merged_dataset[f'{col}_has_official_word'] = merged_dataset.apply(
            lambda row: check_words_in_url(row['OfficialName_cleaned'], row[col]), axis=1).astype(int)

        # Check if 'Abbreviation' is in the URL
        merged_dataset[f'{col}_has_abbreviation'] = merged_dataset.apply(
            lambda row: check_abbreviation_in_url(row['Abbreviation'], row[col]), axis=1).astype(int)

merged_dataset['OfficialName_cleaned'] = merged_dataset['OfficialName_cleaned'].str.replace(' ', '', regex=True) # Remove spaces
# Calculate the length of 'OfficialName_cleaned'
merged_dataset['OfficialName_cleaned_length'] = merged_dataset['OfficialName_cleaned'].str.len()

# Calculate the length of 'Abbreviation'
merged_dataset['Abbreviation_length'] = merged_dataset['Abbreviation'].str.len()

## Check if shapes are still the same after merging

In [ ]:
# Load data and preprocess domains
dataset_query_path = 'dataset_incl_query.csv'
search_results_paths = ['search_results_DDG.csv']
original_dataset = load_data(dataset_query_path, search_results_paths)


# Print the shape of the original dataset
print("Original Dataset Shape:", original_dataset.shape)

# Lowercase all URL columns before processing
url_columns = ['URL'] + [f'URL{i}' for i in range(1, 6)]
for col in url_columns:
    original_dataset[col] = original_dataset[col].str.lower()

# Initialize 'OfficialName_cleaned' and perform cleaning operations
original_dataset['OfficialName_cleaned'] = original_dataset['OfficialName'].str.lower()
original_dataset['OfficialName_cleaned'] = original_dataset['OfficialName_cleaned'].str.replace(r"\[.*?\]", "", regex=True)
original_dataset['OfficialName_cleaned'] = original_dataset['OfficialName_cleaned'].str.replace(r"\(.*?\)", "", regex=True)
original_dataset['OfficialName_cleaned'] = original_dataset['OfficialName_cleaned'].str.replace('-', '', regex=True)
original_dataset['Abbreviation'] = original_dataset.apply(
    lambda row: create_abbreviation(row['OfficialName_cleaned']) if pd.isna(row['Abbreviation']) else row['Abbreviation'],
    axis=1
    )

# Preprocess the domains to extract the core domain
preprocess_domains(original_dataset, url_columns)

# Extract and append domain features for each URL column with a suffix indicating the column
for col in url_columns:
    original_dataset[f'{col}_clean_domain'] = original_dataset[col].apply(clean_extract_domain)

# Compare domains and add the comparison results to the DataFrame
comparison_domain_cols = [f'{col}_clean_domain' for col in url_columns if col != 'URL']
original_dataset['domain_matches'] = original_dataset.apply(
    lambda row: compare_domains(row, 'URL_clean_domain', comparison_domain_cols), axis=1
)

# Calculate the length of each 'URL(i)_domain'
for col in url_columns_without_official:
    domain_col = f'{col}_domain'
    if domain_col in original_dataset.columns:
        original_dataset[f'{col}_core_domain'] = original_dataset[domain_col].apply(get_domain_without_tld)
        original_dataset[f'{col}_domain_length'] = original_dataset[f'{col}_core_domain'].str.len()

# Calculate additional features
original_dataset['Abbreviation'] = original_dataset['Abbreviation'].str.lower()
original_dataset['OfficialName_cleaned'] = original_dataset['OfficialName_cleaned'].str.replace(' ', '', regex=True)
original_dataset['OfficialName_cleaned_length'] = original_dataset['OfficialName_cleaned'].str.len()
original_dataset['Abbreviation_length'] = original_dataset['Abbreviation'].str.len()

# Print the shape of the merged dataset
print("Merged Dataset Shape:", original_dataset.shape)


## Print statements for checking


In [ ]:
# Display results and optionally save the updated dataset
print(merged_dataset[['OfficialName','Abbreviation','URL_domain','URL1_domain', 'URL1_clean_domain','OfficialName_cleaned_length','Abbreviation_length','URL', 'URL1','URL2','URL3','URL4','URL5', 'URL1_domain_length', 'URL1_has_official_word','URL1_has_abbreviation', 'domain_matches']].head(42))
print(merged_dataset.info())

## Extract tld for deep cleaning domains

In [ ]:
import pandas as pd
import re

def extract_tlds(url):
    """ Extracts the TLD from a URL."""
    if pd.isna(url):
        return None
    match = re.search(r'\.\w+($|\s|\/)', url)
    if match:
        return match.group(0).strip().strip('/')
    return None

# List to collect all TLDs
all_tlds = set()

# Columns to check
url_columns = [f'URL{i}_clean_domain' for i in range(1, 6)]

# Loop through each column, apply the function, and update the set of TLDs
for col in url_columns:
    # Apply the extract_tlds function to the column
    current_tlds = merged_dataset[col].apply(extract_tlds)
    # Drop None values and update all_tlds set
    all_tlds.update(tld for tld in current_tlds if tld is not None)

# Print the unique TLDs found
print("Unique TLDs found in URL columns:", all_tlds)


### Count missing values and replace with NaN

In [ ]:
print(merged_dataset['OfficialName_cleaned'].isna().sum())
print(merged_dataset['Abbreviation'].isna().sum())


In [ ]:
# Calculating the number of missing values in each column
missing_values_count = merged_dataset.isna().sum()

# Filtering and printing only the columns that have missing values
columns_with_na = missing_values_count[missing_values_count > 0]
print(columns_with_na)

# Calculating the percentage of missing values in each column
total_rows = len(merged_dataset)
missing_percentage = (merged_dataset.isna().sum() / total_rows) * 100

# Displaying the percentage of missing values for each column
print(missing_percentage)


In [ ]:
merged_dataset = merged_dataset.fillna('NaN')

## Remove tld for better comparision between scraped and true URL

In [ ]:
import re
import pandas as pd

# List of TLDs to exclude
tlds = all_tlds

# Compile regex pattern outside the function for efficiency
tld_pattern = r'\.(' + '|'.join([re.escape(tld.strip('.')) for tld in tlds]) + ')$'

def extract_until_tld(url):
    """Removes the last TLD from the URL if it matches the predefined list."""
    if pd.isna(url):
        return ""  # Handle NaN values
    # Replace the last occurrence of TLD
    url = re.sub(tld_pattern, "", url)
    return url


url_columns = [f'URL{i}_clean_domain' for i in range(1, 6)]


for col in url_columns:
    merged_dataset[f'{col}_before_tld'] = merged_dataset[col].apply(extract_until_tld)
print(merged_dataset[['URL1_clean_domain_before_tld','URL2_clean_domain_before_tld',
                      'URL3_clean_domain_before_tld','URL4_clean_domain_before_tld',
                      'URL5_clean_domain_before_tld']].head(10))


# Feature engineering

## Calculate jaccard similarity, check subsequence, sequence match score and levenshtein distance

In [ ]:
%pip install pandarallel
%pip install python-Levenshtein

import pandas as pd
from difflib import SequenceMatcher
from pandarallel import pandarallel
import Levenshtein
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def jaccard_similarity(set1, set2):
    """ Calculate Jaccard similarity score. """
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

def check_subsequence(company, url):
    """ Check if all characters in the company name can be found in sequence in the URL. """
    it = iter(url)
    return all(char in it for char in company)

def sequence_match_score(a, b):
    """ Use SequenceMatcher to find how similar two strings are. """
    return SequenceMatcher(None, a, b).ratio()

def levenshtein_distance_score(a, b):
    """ Calculate the Levenshtein distance between two strings. """
    return Levenshtein.distance(a, b)

def safe_set_conversion(text):
    """ Convert a text string to a set of characters, handling None safely. """
    if not text:
        return set()
    return set(text)

def cosine_similarity_score(a, b):
    """ Calculate the cosine similarity between two strings. """
    if not a or not b:
        return 0.0

    if len(a) < 3 or len(b) < 3:
        return 0.0  # Strings are too short for meaningful comparison

    vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 3))
    vectors = vectorizer.fit_transform([a, b])

    if vectors.shape[1] == 0:
        return 0.0  # No features extracted, possibly due to containing only stop words

    similarity = cosine_similarity(vectors)
    return similarity[0, 1]


def hamming_distance_score(a, b):
    """ Calculate the Hamming distance between two strings. """
    return sum(ch1 != ch2 for ch1, ch2 in zip(a, b))

def ngram_overlap_score(a, b):
    """ Calculate the n-gram overlap between two strings. """
    n = 3  # Adjust n-gram size as needed
    a_ngrams = set([a[i:i+n] for i in range(len(a)-n+1)])
    b_ngrams = set([b[i:i+n] for i in range(len(b)-n+1)])

    # Check for zero denominator
    if len(a_ngrams) == 0 or len(b_ngrams) == 0:
        return 0.0

    overlap = len(a_ngrams.intersection(b_ngrams))
    return overlap / min(len(a_ngrams), len(b_ngrams))


url_columns_without_tld = [f'URL{i}_clean_domain_before_tld' for i in range(1, 6)]

# Initialization
pandarallel.initialize(progress_bar=True)

# Parallel application of functions
for col in url_columns_without_tld:
    merged_dataset[f'{col}_official_jaccard'] = merged_dataset.parallel_apply(
        lambda row: jaccard_similarity(safe_set_conversion(row['OfficialName']), safe_set_conversion(row[col])), axis=1)

    merged_dataset[f'{col}_abbrev_jaccard'] = merged_dataset.parallel_apply(
        lambda row: jaccard_similarity(safe_set_conversion(row['Abbreviation']), safe_set_conversion(row[col])), axis=1)

    merged_dataset[f'{col}_official_is_subsequence'] = merged_dataset.parallel_apply(
        lambda row: check_subsequence(row['OfficialName'], row[col]), axis=1).astype(int)

    merged_dataset[f'{col}_abbrev_is_subsequence'] = merged_dataset.parallel_apply(
        lambda row: check_subsequence(row['Abbreviation'], row[col]), axis=1).astype(int)

    merged_dataset[f'{col}_official_seq_match'] = merged_dataset.parallel_apply(
        lambda row: sequence_match_score(row['OfficialName'], row[col]), axis=1)

    merged_dataset[f'{col}_abbrev_seq_match'] = merged_dataset.parallel_apply(
        lambda row: sequence_match_score(row['Abbreviation'], row[col]), axis=1)

    # Applying Levenshtein distance calculations
    merged_dataset[f'{col}_official_levenshtein'] = merged_dataset.parallel_apply(
        lambda row: levenshtein_distance_score(row['OfficialName'], row[col]), axis=1)

    merged_dataset[f'{col}_abbrev_levenshtein'] = merged_dataset.parallel_apply(
        lambda row: levenshtein_distance_score(row['Abbreviation'], row[col]), axis=1)

    merged_dataset[f'{col}_official_cosine_similarity'] = merged_dataset.parallel_apply(
        lambda row: cosine_similarity_score(row['OfficialName'], row[col]), axis=1)

    merged_dataset[f'{col}_abbrev_cosine_similarity'] = merged_dataset.parallel_apply(
        lambda row: cosine_similarity_score(row['Abbreviation'], row[col]), axis=1)

    merged_dataset[f'{col}_hamming_distance'] = merged_dataset.parallel_apply(
        lambda row: hamming_distance_score(row['OfficialName'], row[col]), axis=1)

    merged_dataset[f'{col}_ngram_overlap'] = merged_dataset.parallel_apply(
        lambda row: ngram_overlap_score(row['OfficialName'], row[col]), axis=1)

columns_to_display = [
    'EntityNumber', 'OfficialName', 'Abbreviation',
    'URL1_clean_domain_before_tld',
    'URL1_clean_domain_before_tld_official_jaccard', 'URL1_clean_domain_before_tld_abbrev_jaccard',
    'URL1_clean_domain_before_tld_official_is_subsequence', 'URL1_clean_domain_before_tld_abbrev_is_subsequence',
    'URL1_clean_domain_before_tld_official_seq_match', 'URL1_clean_domain_before_tld_abbrev_seq_match',
    'URL1_clean_domain_before_tld_official_levenshtein', 'URL1_clean_domain_before_tld_abbrev_levenshtein',
    'URL1_clean_domain_before_tld_official_cosine_similarity', 'URL1_clean_domain_before_tld_abbrev_cosine_similarity',
    'URL1_clean_domain_before_tld_hamming_distance','URL1_clean_domain_before_tld_ngram_overlap'
]
# Print the first few rows of these columns to inspect
print("Validation of DataFrame calculations:")
print(merged_dataset[columns_to_display].head(30))


contains_abbrev_subseq_count = merged_dataset['URL1_clean_domain_before_tld_abbrev_is_subsequence'].sum()
print(f"\nNumber of URLs where 'Abbreviation' is a subsequence: {contains_abbrev_subseq_count}")
contains_Official_subseq_count = merged_dataset['URL1_clean_domain_before_tld_official_is_subsequence'].sum()
print(f"Number of URLs where 'OfficialName' is a subsequence: {contains_Official_subseq_count}")

print("\nData Types and NaN Presence Check:")
print(merged_dataset[columns_to_display].info())


print("\nStatistical Summary of Computational Fields:")
print(merged_dataset[[col for col in merged_dataset.columns if 'jaccard' in col or 'seq_match' in col]].describe())

In [ ]:
print(merged_dataset.columns)

In [ ]:
print(len(merged_dataset))

17239


### URL embedding

In [ ]:
from transformers import BertTokenizer, AutoConfig, AutoModelForMaskedLM
import torch
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer

# URLBERT: https://github.com/Davidup1/URLBERT

# Initialize tokenizer and model configuration
tokenizer = BertTokenizer(vocab_file="URLBERT-master/bert_tokenizer/vocab.txt")
config_kwargs = {
    "hidden_dropout_prob": 0.2,
    "vocab_size": 5000
}
config = AutoConfig.from_pretrained("URLBERT-master/bert_config", **config_kwargs)
bert_model = AutoModelForMaskedLM.from_config(config=config)
bert_model.resize_token_embeddings(config_kwargs["vocab_size"])

# Load pretrained model weights
bert_dict = torch.load("URLBERT-master/bert_model/urlBERT.pt", map_location='cpu')
bert_model.load_state_dict(bert_dict)
bert_model.eval()
bert_model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Function to generate embeddings
def embed_text_in_batches(texts, batch_size=10):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Batch Processing"):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
        inputs = {k: v.to(bert_model.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs, output_hidden_states=True)
            last_hidden_state = outputs.hidden_states[-1][:, 0, :].cpu().numpy()
            all_embeddings.append(last_hidden_state)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    return np.concatenate(all_embeddings, axis=0)

# Generate embeddings for URLs
url_columns = [f"URL{i}" for i in range(1, 6)]
urls = [merged_dataset[col].tolist() for col in url_columns]
all_urls = [url for sublist in urls for url in sublist]  # Flatten list of URLs
url_embeddings = embed_text_in_batches(all_urls, batch_size=10)

# Reshape embeddings to match the number of samples and URLs
n_samples = len(merged_dataset)
n_urls = len(url_columns)
embed_dim = url_embeddings.shape[1]  # Dimension of each URL embedding
url_embeddings = url_embeddings.reshape(n_samples, n_urls * embed_dim)

# Data preparation, stratified splitting, scaling, training, predictiong and evaluation of model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, jaccard_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler

# Prepare the data
X = merged_dataset[['URL1_has_official_word', 'URL1_has_abbreviation',
                    'URL2_has_official_word', 'URL2_has_abbreviation',
                    'URL3_has_official_word', 'URL3_has_abbreviation',
                    'URL4_has_official_word', 'URL4_has_abbreviation',
                    'URL5_has_official_word', 'URL5_has_abbreviation',
                    'OfficialName_cleaned_length', 'Abbreviation_length',
                    'URL1_clean_domain_before_tld_official_jaccard',
                    'URL1_clean_domain_before_tld_abbrev_jaccard',
                    'URL1_clean_domain_before_tld_official_is_subsequence',
                    'URL1_clean_domain_before_tld_abbrev_is_subsequence',
                    'URL1_clean_domain_before_tld_official_seq_match',
                    'URL1_clean_domain_before_tld_abbrev_seq_match',
                    'URL1_clean_domain_before_tld_official_levenshtein',
                    'URL1_clean_domain_before_tld_abbrev_levenshtein',
                    'URL1_clean_domain_before_tld_official_cosine_similarity',
                    'URL1_clean_domain_before_tld_abbrev_cosine_similarity',
                    'URL1_clean_domain_before_tld_hamming_distance',
                    'URL1_clean_domain_before_tld_ngram_overlap',
                    'URL2_clean_domain_before_tld_official_jaccard',
                    'URL2_clean_domain_before_tld_abbrev_jaccard',
                    'URL2_clean_domain_before_tld_official_is_subsequence',
                    'URL2_clean_domain_before_tld_abbrev_is_subsequence',
                    'URL2_clean_domain_before_tld_official_seq_match',
                    'URL2_clean_domain_before_tld_abbrev_seq_match',
                    'URL2_clean_domain_before_tld_official_levenshtein',
                    'URL2_clean_domain_before_tld_abbrev_levenshtein',
                    'URL2_clean_domain_before_tld_official_cosine_similarity',
                    'URL2_clean_domain_before_tld_abbrev_cosine_similarity',
                    'URL2_clean_domain_before_tld_hamming_distance',
                    'URL2_clean_domain_before_tld_ngram_overlap',
                    'URL3_clean_domain_before_tld_official_jaccard',
                    'URL3_clean_domain_before_tld_abbrev_jaccard',
                    'URL3_clean_domain_before_tld_official_is_subsequence',
                    'URL3_clean_domain_before_tld_abbrev_is_subsequence',
                    'URL3_clean_domain_before_tld_official_seq_match',
                    'URL3_clean_domain_before_tld_abbrev_seq_match',
                    'URL3_clean_domain_before_tld_official_levenshtein',
                    'URL3_clean_domain_before_tld_abbrev_levenshtein',
                    'URL3_clean_domain_before_tld_official_cosine_similarity',
                    'URL3_clean_domain_before_tld_abbrev_cosine_similarity',
                    'URL3_clean_domain_before_tld_hamming_distance',
                    'URL3_clean_domain_before_tld_ngram_overlap',
                    'URL4_clean_domain_before_tld_official_jaccard',
                    'URL4_clean_domain_before_tld_abbrev_jaccard',
                    'URL4_clean_domain_before_tld_official_is_subsequence',
                    'URL4_clean_domain_before_tld_abbrev_is_subsequence',
                    'URL4_clean_domain_before_tld_official_seq_match',
                    'URL4_clean_domain_before_tld_abbrev_seq_match',
                    'URL4_clean_domain_before_tld_official_levenshtein',
                    'URL4_clean_domain_before_tld_abbrev_levenshtein',
                    'URL4_clean_domain_before_tld_official_cosine_similarity',
                    'URL4_clean_domain_before_tld_abbrev_cosine_similarity',
                    'URL4_clean_domain_before_tld_hamming_distance',
                    'URL4_clean_domain_before_tld_ngram_overlap',
                    'URL5_clean_domain_before_tld_official_jaccard',
                    'URL5_clean_domain_before_tld_abbrev_jaccard',
                    'URL5_clean_domain_before_tld_official_is_subsequence',
                    'URL5_clean_domain_before_tld_abbrev_is_subsequence',
                    'URL5_clean_domain_before_tld_official_seq_match',
                    'URL5_clean_domain_before_tld_abbrev_seq_match',
                    'URL5_clean_domain_before_tld_official_levenshtein',
                    'URL5_clean_domain_before_tld_abbrev_levenshtein',
                    'URL5_clean_domain_before_tld_official_cosine_similarity',
                    'URL5_clean_domain_before_tld_abbrev_cosine_similarity',
                    'URL5_clean_domain_before_tld_hamming_distance',
                    'URL5_clean_domain_before_tld_ngram_overlap']]

X = np.hstack((X, url_embeddings))

# Encode multilabel target
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(merged_dataset['domain_matches'])

# Split the data using stratified splitting
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Verify Encoding, Verify Stratified Splitting, Check Data Scaling, Evaluate Model Outputs and Debug Total Support

In [ ]:
import numpy as np

# Check the shape of the encoded target variable
print("Shape of encoded targets:", y_encoded.shape)

# Check unique label counts
print("Number of unique labels:", len(mlb.classes_))

# Check a few rows to ensure encoding looks correct
print("Sample encoded labels:", y_encoded[:5])

# Check distribution of labels in the full dataset and in the split datasets
print("Label distribution in full dataset:", np.sum(y_encoded, axis=0))
print("Label distribution in training set:", np.sum(y_train, axis=0))
print("Label distribution in test set:", np.sum(y_test, axis=0))

# Check first few rows of scaled features
print("Sample scaled training features:", X_train_scaled[:5])
print("Sample scaled test features:", X_test_scaled[:5])

# Calculate total support manually
total_support = np.sum(y_test, axis=0)
print("Manual calculation of total support:", total_support)
print("Sum of total supports:", np.sum(total_support))

# Verify total support matches manual calculation
assert np.array_equal(total_support, np.sum(y_test, axis=0))


### Logistic Regression


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, jaccard_score
from sklearn.linear_model import LogisticRegression

# Train logistic regression model
logistic_model = MultiOutputClassifier(LogisticRegression(max_iter=3000))
logistic_model.fit(X_train_scaled, y_train)



# Convert mlb.classes_ to a list of strings
target_names = [str(label) for label in mlb.classes_]

# Evaluate the models
print("Logistic Regression Model Evaluation:")
logistic_pred = logistic_model.predict(X_test_scaled)
print(classification_report(y_test, logistic_pred, target_names=target_names))



# Accuracy (note: this might not be very informative in multi-label settings)
print("Accuracy - Logistic Regression:", accuracy_score(y_test, logistic_pred))


# Hamming Loss
print("Hamming Loss - Logistic Regression:", hamming_loss(y_test, logistic_pred))


# Jaccard Score
# Calculate Jaccard Score for each label as an average
print("Jaccard Score - Logistic Regression:", jaccard_score(y_test, logistic_pred, average='samples'))


##### Most important feature

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Retrieve the coefficients from each classifier in the MultiOutputClassifier
coefs = [estimator.coef_ for estimator in logistic_model.estimators_]

# Average the absolute coefficients across all classifiers (you could also choose max, min, etc.)
average_coefs = np.mean([np.abs(coef[0]) for coef in coefs], axis=0)

# Map feature names to average absolute coefficients
feature_names = X.columns
feature_coefficients_map = dict(zip(feature_names, average_coefs))

# Sort the features based on their average absolute coefficients
sorted_features = sorted(feature_coefficients_map.items(), key=lambda x: x[1], reverse=True)

# Limit to top 10 features
top_features = sorted_features[:10]

# Print the sorted list of top 10 features and their coefficients
for feature, coefficient in top_features:
    print(f"Feature: {feature}, Coefficient: {coefficient}")

# Extract features and coefficients for plotting the top 10
features = [x[0] for x in top_features]
coefficients = [x[1] for x in top_features]

# Creating a bar plot for the top 10 features
plt.figure(figsize=(10, 6))
plt.barh(features, coefficients)
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Top 10 Feature Importances')
plt.gca().invert_yaxis()  # To display the highest values at the top
plt.savefig('Most_imp_feat_LG')
plt.show()


### Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, jaccard_score

# Train decision tree model using MultiOutputClassifier
tree_model = MultiOutputClassifier(DecisionTreeClassifier())
tree_model.fit(X_train_scaled, y_train)

print("\nDecision Tree Model Evaluation:")
tree_pred = tree_model.predict(X_test_scaled)

print(classification_report(y_test, tree_pred, target_names=target_names))

print("Accuracy - Decision Tree:", accuracy_score(y_test, tree_pred))

print("Hamming Loss - Decision Tree:", hamming_loss(y_test, tree_pred))

print("Jaccard Score - Decision Tree:", jaccard_score(y_test, tree_pred, average='samples'))

##### Feature importance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Retrieve the feature importances from each classifier in the MultiOutputClassifier
importances = [estimator.feature_importances_ for estimator in tree_model.estimators_]

# Average the feature importances across all classifiers
average_importances = np.mean(importances, axis=0)

# Map feature names to average importances
feature_names = X.columns
feature_importance_map = dict(zip(feature_names, average_importances))

# Sort the features based on their average importances
sorted_features = sorted(feature_importance_map.items(), key=lambda x: x[1], reverse=True)

# Limit to top 10 features
top_features = sorted_features[:10]

# Print the sorted list of top 10 features and their importances
for feature, importance in top_features:
    print(f"Feature: {feature}, Importance: {importance}")

# Extract features and importances for plotting the top 10
features = [x[0] for x in top_features]
importances = [x[1] for x in top_features]

# Creating a bar plot for the top 10 features
plt.figure(figsize=(10, 6))
plt.barh(features, importances)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 10 Feature Importances - Decision Tree')
plt.gca().invert_yaxis()  # To display the highest values at the top
plt.savefig('Most_imp_feat_DT')
plt.show()


### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, jaccard_score

# Train Random Forest model using MultiOutputClassifier
rf_model = MultiOutputClassifier(RandomForestClassifier())
rf_model.fit(X_train_scaled, y_train)

# Evaluate Random Forest model
rf_pred = rf_model.predict(X_test_scaled)
print("Random Forest Model Evaluation:")
print(classification_report(y_test, rf_pred, target_names=target_names))

# Accuracy (note: this might not be very informative in multi-label settings)
print("Accuracy - Random Forest:", accuracy_score(y_test, rf_pred))

# Hamming Loss
print("Hamming Loss - Random Forest:", hamming_loss(y_test, rf_pred))

# Jaccard Score
# Calculate Jaccard Score for each label as an average
print("Jaccard Score - Random Forest:", jaccard_score(y_test, rf_pred, average='samples'))

##### Feature importance

In [ ]:
# Retrieve the feature importances from each classifier in the MultiOutputClassifier
importances = [estimator.feature_importances_ for estimator in rf_model.estimators_]

# Average the feature importances across all classifiers
average_importances = np.mean(importances, axis=0)

# Map feature names to average importances
feature_names = X.columns
feature_importance_map = dict(zip(feature_names, average_importances))

# Sort the features based on their average importances
sorted_features = sorted(feature_importance_map.items(), key=lambda x: x[1], reverse=True)

# Limit to top 10 features
top_features = sorted_features[:10]

# Print the sorted list of top 10 features and their importances
for feature, importance in top_features:
    print(f"Feature: {feature}, Importance: {importance}")

# Extract features and importances for plotting the top 10
features = [x[0] for x in top_features]
importances = [x[1] for x in top_features]

# Creating a bar plot for the top 10 features
plt.figure(figsize=(10, 6))
plt.barh(features, importances)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 10 Feature Importances - Random Forest')
plt.gca().invert_yaxis()  # To display the highest values at the top
plt.savefig('Most_imp_feat_RF')
plt.show()


### Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, jaccard_score

# Train Gradient Boosting model using MultiOutputClassifier
gb_model = MultiOutputClassifier(GradientBoostingClassifier())
gb_model.fit(X_train_scaled, y_train)

# Evaluate Gradient Boosting model
gb_pred = gb_model.predict(X_test_scaled)
print("\nGradient Boosting Model Evaluation:")
print(classification_report(y_test, gb_pred, target_names=target_names))

# Accuracy (note: this might not be very informative in multi-label settings)
print("Accuracy - Gradient Boosting Classifier:", accuracy_score(y_test, gb_pred))

# Hamming Loss
print("Hamming Loss - Gradient Boosting Classifier:", hamming_loss(y_test, gb_pred))

# Jaccard Score
# Calculate Jaccard Score for each label as an average
print("Jaccard Score - Gradient Boosting Classifier:", jaccard_score(y_test, gb_pred, average='samples'))

#### Feature importance

In [ ]:
# Retrieve the feature importances from each classifier in the MultiOutputClassifier
importances = [estimator.feature_importances_ for estimator in gb_model.estimators_]

# Average the feature importances across all classifiers
average_importances = np.mean(importances, axis=0)

# Map feature names to average importances
feature_names = X.columns
feature_importance_map = dict(zip(feature_names, average_importances))

# Sort the features based on their average importances
sorted_features = sorted(feature_importance_map.items(), key=lambda x: x[1], reverse=True)

# Limit to top 10 features
top_features = sorted_features[:10]

# Print the sorted list of top 10 features and their importances
for feature, importance in top_features:
    print(f"Feature: {feature}, Importance: {importance}")

# Extract features and importances for plotting the top 10
features = [x[0] for x in top_features]
importances = [x[1] for x in top_features]

# Creating a bar plot for the top 10 features
plt.figure(figsize=(10, 6))
plt.barh(features, importances)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 10 Feature Importances - Gradient Boosting')
plt.gca().invert_yaxis()  # To display the highest values at the top
plt.savefig('Most_imp_feat_GB')
plt.show()


### Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, jaccard_score

# Train SVM model using MultiOutputClassifier
svm_model = MultiOutputClassifier(SVC())
svm_model.fit(X_train_scaled, y_train)

# Evaluate SVM model
svm_pred = svm_model.predict(X_test_scaled)
print("\nSVM Model Evaluation:")
print(classification_report(y_test, svm_pred, target_names=target_names))

# Accuracy (note: this might not be very informative in multi-label settings)
print("Accuracy - Support Vector Machine (SVM):", accuracy_score(y_test, svm_pred))

# Hamming Loss
print("Hamming Loss - Support Vector Machine (SVM):", hamming_loss(y_test, svm_pred))

# Jaccard Score
# Calculate Jaccard Score for each label as an average
print("Jaccard Score - Support Vector Machine (SVM):", jaccard_score(y_test, svm_pred, average='samples'))

### Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, jaccard_score

# Train Neural Network model using MultiOutputClassifier
nn_model = MultiOutputClassifier(MLPClassifier())
nn_model.fit(X_train_scaled, y_train)

# Evaluate Neural Network model
nn_pred = nn_model.predict(X_test_scaled)
print("\nNeural Network Model Evaluation:")
print(classification_report(y_test, nn_pred, target_names=target_names))

# Accuracy (note: this might not be very informative in multi-label settings)
print("Accuracy - Neural Network:", accuracy_score(y_test, nn_pred))

# Hamming Loss
print("Hamming Loss - Neural Network:", hamming_loss(y_test, nn_pred))

# Jaccard Score
# Calculate Jaccard Score for each label as an average
print("Jaccard Score - Neural Network:", jaccard_score(y_test, nn_pred, average='samples'))

#### Shap for explaining model predictions

In [ ]:
%pip install shap
import shap

# Create a SHAP explainer and calculate SHAP values
explainer = shap.KernelExplainer(nn_model.predict, shap.sample(X_train_scaled, 100))  # using 100 samples for approximation
shap_values = explainer.shap_values(shap.sample(X_test_scaled, 100))

# Plot the summary of SHAP values
shap.summary_plot(shap_values, shap.sample(X_test_scaled, 100), feature_names=X.columns)
plt.savefig('shap_summary_plot.png')
plt.close()

### K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, jaccard_score

# Train KNN model using MultiOutputClassifier
knn_model = MultiOutputClassifier(KNeighborsClassifier())
knn_model.fit(X_train_scaled, y_train)

# Evaluate KNN model
knn_pred = knn_model.predict(X_test_scaled)
print("\nK-Nearest Neighbors Model Evaluation:")
print(classification_report(y_test, knn_pred, target_names=target_names))

# Accuracy (note: this might not be very informative in multi-label settings)
print("Accuracy - K-Nearest Neighbors:", accuracy_score(y_test, knn_pred))

# Hamming Loss
print("Hamming Loss - K-Nearest Neighbors:", hamming_loss(y_test, knn_pred))

# Jaccard Score
# Calculate Jaccard Score for each label as an average
print("Jaccard Score - K-Nearest Neighbors:", jaccard_score(y_test, knn_pred, average='samples'))

### Manual Visualization

In [ ]:
import pandas as pd

# Convert logistic predictions into DataFrame
pred_df = pd.DataFrame(rf_pred, columns=[f'Label_{i+1}' for i in range(rf_pred.shape[1])], index=X_test.index)

# Adjusted function to handle non-string label types
def extract_predicted_labels(row):
    labels = []
    for i, value in enumerate(row):
        if value == 1:
            labels.append(str(mlb.classes_[i]))  # Convert label to string
    return ', '.join(labels) if labels else 'No Label Predicted'

# Apply function to rows to create a 'PredictedLabel' column
pred_df['PredictedLabel'] = pred_df.apply(extract_predicted_labels, axis=1)

# Join this with the entity number from the original dataset

final_df = merged_dataset.loc[pred_df.index, ['EntityNumber']].join(pred_df[['PredictedLabel']])

# Here is the final DataFrame
print(final_df.head(50))

# Optional: Save this DataFrame to CSV
# final_df.to_csv('predictions_with_entity_number.csv')


# Use predictions to return correct URL

## Wrongly appointed labels

In [ ]:
import pandas as pd
import numpy as np

# Convert predictions and actuals to readable label lists
predicted_labels = ['; '.join(str(label) for label in labels) if labels else 'No Labels Predicted' for labels in mlb.inverse_transform(logistic_pred)]
actual_labels = ['; '.join(str(label) for label in labels) if labels else 'No Labels Predicted' for labels in mlb.inverse_transform(y_test)]

# Step 1: Calculate Incorrect Predictions
# Create a boolean array where True indicates a mismatch between prediction and actual label
incorrect_predictions_mask = logistic_pred != y_test

# Step 2: Count Incorrect Predictions
# Sum the True values in incorrect_predictions_mask for each sample
incorrect_counts = incorrect_predictions_mask.sum(axis=1)

# Print total number of incorrect label predictions across all samples
total_incorrect_labels = incorrect_counts.sum()
print("Total incorrect label predictions:", total_incorrect_labels)

# Step 3: Create a DataFrame for incorrect predictions
# Include only those samples that have one or more incorrectly predicted labels
samples_with_incorrect_predictions = incorrect_counts > 0

# Convert the lists to Series for proper indexing
predicted_labels_series = pd.Series(predicted_labels, index=X_test.index)
actual_labels_series = pd.Series(actual_labels, index=X_test.index)

# Extract the corresponding rows from X_test

incorrect_predictions_df = pd.DataFrame({
    'EntityNumber': merged_dataset.loc[X_test.index[samples_with_incorrect_predictions], 'EntityNumber'],
    'IncorrectCount': incorrect_counts[samples_with_incorrect_predictions],
    'PredictedLabels': predicted_labels_series[samples_with_incorrect_predictions],
    'ActualLabels': actual_labels_series[samples_with_incorrect_predictions]
})

# Print or explore the DataFrame
print(incorrect_predictions_df.head(20))

# Optional: Save this DataFrame to a CSV file
# incorrect_predictions_df.to_csv('incorrect_predictions.csv')



# Hyperparameter tuning

### Logistic Regression

In [ ]:
%pip install iterative-stratification

import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Parameters for GridSearchCV
param_grid = {
    'estimator__C': [0.1, 1, 10],
    'estimator__penalty': ['l2'],
    'estimator__solver': ['liblinear']
}

# Create the logistic regression model within a MultiOutputClassifier
logistic_model = MultiOutputClassifier(LogisticRegression(max_iter=3000))

# Setup stratified cross-validation folds for multi-label data
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define a scorer, example using f1_score with micro averaging
f1_macro_scorer = make_scorer(f1_score, average='macro')

# Initialize GridSearchCV with multi-label stratified folds and f1_micro as the scoring function
grid_search = GridSearchCV(logistic_model, param_grid, scoring=f1_macro_scorer, cv=mskf, verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train_scaled, y_train)

# Best estimator and score found by GridSearchCV
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

df_new_results = pd.DataFrame([grid_search.best_params_], index=[0])
df_new_results['model'] = 'Logistic Regression'

# Check if the results CSV file exists
if os.path.exists('hyperparameter_tuning_results.csv'):
    existing_results = pd.read_csv('hyperparameter_tuning_results.csv')
    combined_results = pd.concat([existing_results, df_new_results], ignore_index=True)
else:
    combined_results = df_new_results

# Save the updated results back to CSV
combined_results.to_csv('hyperparameter_tuning_results.csv', index=False)



### Decision Tree

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import GridSearchCV
import os

# Parameters for GridSearchCV
param_grid_dt = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10]
}

# Setup stratified cross-validation folds for multi-label data
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Decision tree model
decision_tree_model = DecisionTreeClassifier(random_state=42)

# Use f1_score with micro averaging as the scoring function
f1_macro_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV setup
grid_search_dt = GridSearchCV(decision_tree_model, param_grid_dt, cv=mskf, scoring=f1_macro_scorer, verbose=1)
grid_search_dt.fit(X_train, y_train)

# Evaluate and print best parameters and model performance
print("Best parameters:", grid_search_dt.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search_dt.best_score_))

# Create a DataFrame from the best parameters and add a column specifying the model
df_new_results = pd.DataFrame([grid_search_dt.best_params_], index=[0])
df_new_results['model'] = 'Decision Tree'

# Check if the results CSV file exists
if os.path.exists('hyperparameter_tuning_results.csv'):
    existing_results = pd.read_csv('hyperparameter_tuning_results.csv')
    combined_results = pd.concat([existing_results, df_new_results], ignore_index=True)
else:
    combined_results = df_new_results

# Save the updated results back to CSV
combined_results.to_csv('hyperparameter_tuning_results.csv', index=False)

### Gradient Booster Classifier

In [ ]:
import pandas as pd
import os
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import GridSearchCV

# Parameters for GridSearchCV
param_grid_gb = {
    'estimator__n_estimators': [100, 200],
    'estimator__learning_rate': [0.01, 0.1, 0.2],
    'estimator__max_depth': [3, 5, 8]
}

# Setup stratified cross-validation folds for multi-label data
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Gradient boosting model wrapped in MultiOutputClassifier
gradient_boosting_model = MultiOutputClassifier(GradientBoostingClassifier(random_state=42))

# Use f1_score with micro averaging as the scoring function
f1_macro_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV setup
grid_search_gb = GridSearchCV(gradient_boosting_model, param_grid_gb, cv=mskf, scoring=f1_macro_scorer, verbose=1)
grid_search_gb.fit(X_train, y_train)

# Evaluate and print best parameters and model performance
print("Best parameters:", grid_search_gb.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search_gb.best_score_))

# Create a DataFrame from the best parameters and add a column specifying the model
df_new_results = pd.DataFrame([grid_search_gb.best_params_], index=[0])
df_new_results['model'] = 'Gradient Boosting'

# Check if the results CSV file exists
if os.path.exists('hyperparameter_tuning_results.csv'):
    existing_results = pd.read_csv('hyperparameter_tuning_results.csv')
    combined_results = pd.concat([existing_results, df_new_results], ignore_index=True)
else:
    combined_results = df_new_results

# Save the updated results back to CSV
combined_results.to_csv('hyperparameter_tuning_results.csv', index=False)


### Random Forest

In [ ]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import GridSearchCV

# Parameters for GridSearchCV
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 2, 4]
}

# Setup stratified cross-validation folds for multi-label data
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Random forest model wrapped in MultiOutputClassifier (if needed, otherwise just RandomForestClassifier)
random_forest_model = RandomForestClassifier(random_state=42)

# Use f1_score with micro averaging as the scoring function
f1_macro_scorer = make_scorer(f1_score, average='macro')

# GridSearchCV setup
grid_search_rf = GridSearchCV(random_forest_model, param_grid_rf, cv=mskf, scoring=f1_macro_scorer, verbose=1)
grid_search_rf.fit(X_train, y_train)

# Evaluate and print best parameters and model performance
print("Best parameters:", grid_search_rf.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search_rf.best_score_))

# Create a DataFrame from the best parameters and add a column specifying the model
df_new_results = pd.DataFrame([grid_search_rf.best_params_], index=[0])
df_new_results['model'] = 'Random Forest'

# Check if the results CSV file exists
if os.path.exists('hyperparameter_tuning_results.csv'):
    existing_results = pd.read_csv('hyperparameter_tuning_results.csv')
    combined_results = pd.concat([existing_results, df_new_results], ignore_index=True)
else:
    combined_results = df_new_results

# Save the updated results back to CSV
combined_results.to_csv('hyperparameter_tuning_results.csv', index=False)


### SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# Initialize Multilabel Stratified K-Fold
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up the parameter grid for SVM
param_grid_svm = {
    'estimator__svc__C': [0.1, 1, 10],
    'estimator__svc__gamma': ['scale', 'auto'],
    'estimator__svc__kernel': ['rbf', 'linear']
}

# Create a pipeline that first scales the data then applies SVM
pipeline_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(random_state=42))
])

# Wrap the pipeline and SVM in OneVsRestClassifier
ovr_pipeline = OneVsRestClassifier(pipeline_svm)

# Define the scoring function
f1_macro_scorer = make_scorer(f1_score, average='macro')

# Setup GridSearchCV
grid_search_svm = GridSearchCV(ovr_pipeline, param_grid_svm, cv=mskf, scoring=f1_macro_scorer, verbose=1)
grid_search_svm.fit(X_train, y_train)

# Output the results
print("Best parameters:", grid_search_svm.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search_svm.best_score_))

# Create a DataFrame from the best parameters and add a column specifying the model
df_new_results = pd.DataFrame([grid_search_svm.best_params_], index=[0])
df_new_results['model'] = 'SVM'

# Check if the results CSV file exists
if os.path.exists('hyperparameter_tuning_results.csv'):
    existing_results = pd.read_csv('hyperparameter_tuning_results.csv')
    combined_results = pd.concat([existing_results, df_new_results], ignore_index=True)
else:
    combined_results = df_new_results

# Save the updated results back to CSV
combined_results.to_csv('hyperparameter_tuning_results.csv', index=False)


### Neural Networks

In [ ]:
import tensorflow as tf

def f1_score(y_true, y_pred):
    y_pred_binary = tf.where(y_pred >= 0.5, 1, 0)
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred_binary, 0, 1)))
    possible_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_pred_binary, 0, 1)))

    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1_val

In [ ]:
%pip install keras keras-tuner
import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch
import pandas as pd
import os

def build_model(hp):
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
        keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        keras.layers.Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)),
                  loss='binary_crossentropy',
                  metrics=[f1_score])
    return model

# Create a tuner object
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=3,
    directory='my_dir',
    project_name='keras_tuning'
)

# Execute the hyperparameter search
tuner.search(X_train, y_train, epochs=50, validation_split=0.2)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Prepare the best hyperparameters to be saved
best_params = {
    'model': 'Neural Network',
    'best_units': best_hps.get('units'),
    'best_learning_rate': best_hps.get('learning_rate')
}
df_new_results = pd.DataFrame([best_params])

# Check if the results CSV file exists
results_file = 'hyperparameter_tuning_results.csv'
if os.path.exists(results_file):
    existing_results = pd.read_csv(results_file)
    combined_results = pd.concat([existing_results, df_new_results], ignore_index=True)
else:
    combined_results = df_new_results

# Save the updated results back to CSV
combined_results.to_csv(results_file, index=False)

print(f"Best number of units: {best_hps.get('units')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")



### KNN

In [ ]:
import pandas as pd
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import GridSearchCV

# Parameters for GridSearchCV
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance']
}

# Setup stratified cross-validation folds for multi-label data
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# KNN model wrapped in MultiOutputClassifier (if needed)
knn_model = KNeighborsClassifier()

# Use f1_score with micro averaging as the scoring function
f1_micro_scorer = make_scorer(f1_score, average='micro')

# GridSearchCV setup
grid_search_knn = GridSearchCV(knn_model, param_grid_knn, cv=mskf, scoring=f1_micro_scorer, verbose=1)
grid_search_knn.fit(X_train, y_train)

# Evaluate and print best parameters and model performance
print("Best parameters:", grid_search_knn.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search_knn.best_score_))

# Create a DataFrame from the best parameters and add a column specifying the model
df_new_results = pd.DataFrame([grid_search_knn.best_params_], index=[0])
df_new_results['model'] = 'KNN'

# Check if the results CSV file exists
results_file = 'hyperparameter_tuning_results.csv'
if os.path.exists(results_file):
    existing_results = pd.read_csv(results_file)
    combined_results = pd.concat([existing_results, df_new_results], ignore_index=True)
else:
    combined_results = df_new_results

# Save the updated results back to CSV
combined_results.to_csv(results_file, index=False)


# Predict on unseen data TO DO

In [ ]:
# Load the new dataset
new_data = pd.read_csv('path_to_new_data.csv')

######### First feature creation for new data the same for the training data ########


features = [
    'URL1_has_official_word', 'URL1_has_abbreviation',
    'URL2_has_official_word', 'URL2_has_abbreviation',
    'URL3_has_official_word', 'URL3_has_abbreviation',
    'URL4_has_official_word', 'URL4_has_abbreviation',
    'URL5_has_official_word', 'URL5_has_abbreviation',
    'OfficialName_cleaned_length', 'Abbreviation_length',
    'URL1_clean_domain_before_tld_official_jaccard',
    'URL1_clean_domain_before_tld_abbrev_jaccard',
    'URL1_clean_domain_before_tld_official_is_subsequence',
    'URL1_clean_domain_before_tld_abbrev_is_subsequence',
    'URL1_clean_domain_before_tld_official_seq_match',
    'URL1_clean_domain_before_tld_abbrev_seq_match',
    'URL1_clean_domain_before_tld_official_levenshtein',
    'URL1_clean_domain_before_tld_abbrev_levenshtein',
    'URL2_clean_domain_before_tld_official_jaccard',
    'URL2_clean_domain_before_tld_abbrev_jaccard',
    'URL2_clean_domain_before_tld_official_is_subsequence',
    'URL2_clean_domain_before_tld_abbrev_is_subsequence',
    'URL2_clean_domain_before_tld_official_seq_match',
    'URL2_clean_domain_before_tld_abbrev_seq_match',
    'URL2_clean_domain_before_tld_official_levenshtein',
    'URL2_clean_domain_before_tld_abbrev_levenshtein',
    'URL3_clean_domain_before_tld_official_jaccard',
    'URL3_clean_domain_before_tld_abbrev_jaccard',
    'URL3_clean_domain_before_tld_official_is_subsequence',
    'URL3_clean_domain_before_tld_abbrev_is_subsequence',
    'URL3_clean_domain_before_tld_official_seq_match',
    'URL3_clean_domain_before_tld_abbrev_seq_match',
    'URL3_clean_domain_before_tld_official_levenshtein',
    'URL3_clean_domain_before_tld_abbrev_levenshtein',
    'URL4_clean_domain_before_tld_official_jaccard',
    'URL4_clean_domain_before_tld_abbrev_jaccard',
    'URL4_clean_domain_before_tld_official_is_subsequence',
    'URL4_clean_domain_before_tld_abbrev_is_subsequence',
    'URL4_clean_domain_before_tld_official_seq_match',
    'URL4_clean_domain_before_tld_abbrev_seq_match',
    'URL4_clean_domain_before_tld_official_levenshtein',
    'URL4_clean_domain_before_tld_abbrev_levenshtein',
    'URL5_clean_domain_before_tld_official_jaccard',
    'URL5_clean_domain_before_tld_abbrev_jaccard',
    'URL5_clean_domain_before_tld_official_is_subsequence',
    'URL5_clean_domain_before_tld_abbrev_is_subsequence',
    'URL5_clean_domain_before_tld_official_seq_match',
    'URL5_clean_domain_before_tld_abbrev_seq_match',
    'URL5_clean_domain_before_tld_official_levenshtein',
    'URL5_clean_domain_before_tld_abbrev_levenshtein'
]

# Prepare the new data as per the model's requirements
X_new = new_data[features]


In [ ]:
# Apply the same scaler used on the training data
X_new_scaled = scaler.transform(X_new)  # 'scaler' should be the StandardScaler instance fitted on the training data


In [ ]:
# Make predictions
new_predictions = logistic_model.predict(X_new_scaled)

# Convert these predictions back to label names
new_predicted_labels = mlb.inverse_transform(new_predictions)

# Create a DataFrame to view these predictions clearly
new_predictions_df = pd.DataFrame({
    'EntityNumber': new_data['EntityNumber'],  # Assuming there's an identifier column like EntityNumber
    'PredictedLabels': ['; '.join(labels) if labels else 'No Labels Predicted' for labels in new_predicted_labels]
})

# Display or save the predictions
print(new_predictions_df.head())
# Optional: new_predictions_df.to_csv('new_predictions.csv')


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------